In [17]:
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

warnings.filterwarnings("ignore")
pd.options.display.max_columns = 100

In [2]:
%load_ext watermark

In [3]:
%watermark -p pandas,numpy

pandas : 2.2.2
numpy  : 1.24.3
plotly : 5.22.0
seaborn: 0.13.2



In [4]:
df = pd.read_csv("../data/loan_approval_dataset.csv")

In [5]:
df.head()

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


In [6]:
df.dtypes.value_counts()

int64     10
object     3
Name: count, dtype: int64

In [7]:
df.columns = df.columns.str.strip().tolist()

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
loan_id,4269.0,2.135000e+03,1.232498e+03,1.0,1068.0,2135.0,3202.0,4269.0
no_of_dependents,4269.0,2.498712e+00,1.695910e+00,0.0,1.0,3.0,4.0,5.0
income_annum,4269.0,5.059124e+06,2.806840e+06,200000.0,2700000.0,5100000.0,7500000.0,9900000.0
loan_amount,4269.0,1.513345e+07,9.043363e+06,300000.0,7700000.0,14500000.0,21500000.0,39500000.0
loan_term,4269.0,1.090045e+01,5.709187e+00,2.0,6.0,10.0,16.0,20.0
cibil_score,4269.0,5.999361e+02,1.724304e+02,300.0,453.0,600.0,748.0,900.0
residential_assets_value,4269.0,7.472617e+06,6.503637e+06,-100000.0,2200000.0,5600000.0,11300000.0,29100000.0
commercial_assets_value,4269.0,4.973155e+06,4.388966e+06,0.0,1300000.0,3700000.0,7600000.0,19400000.0
luxury_assets_value,4269.0,1.512631e+07,9.103754e+06,300000.0,7500000.0,14600000.0,21700000.0,39200000.0
bank_asset_value,4269.0,4.976692e+06,3.250185e+06,0.0,2300000.0,4600000.0,7100000.0,14700000.0


In [9]:
df.shape[0] == df["loan_id"].nunique()

True

In [10]:
target = "loan_status"
drop = "loan_id"

In [11]:
cat_cols = [c for c in df.select_dtypes("object").columns.tolist() if c not in target]
num_cols = [c for c in df.select_dtypes(exclude="O").columns.tolist() if c not in drop]

In [12]:
train_cols = [c for c in df.columns if c not in [target, drop]]

In [13]:
df.groupby(target)[num_cols].mean()

,no_of_dependents,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value
loan_status,,,,,,,,,
Approved,2.474774,5.025904e+06,1.524725e+07,10.397590,703.461973,7.399812e+06,5.001355e+06,1.501660e+07,4.959526e+06
Rejected,2.538128,5.113825e+06,1.494606e+07,11.728456,429.468072,7.592498e+06,4.926720e+06,1.530694e+07,5.004960e+06


In [21]:
df[target].value_counts(normalize=True)

loan_status
Approved    0.62216
Rejected    0.37784
Name: proportion, dtype: float64

In [14]:
df.isna().sum()

loan_id                     0
no_of_dependents            0
education                   0
self_employed               0
income_annum                0
loan_amount                 0
loan_term                   0
cibil_score                 0
residential_assets_value    0
commercial_assets_value     0
luxury_assets_value         0
bank_asset_value            0
loan_status                 0
dtype: int64

In [18]:
# https://stackoverflow.com/a/50703596
corr = df.drop(drop, axis=1)[num_cols].corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
# corr[mask] = np.nan
(
    corr.style.background_gradient(cmap="coolwarm", axis=None, vmin=-1, vmax=1)
    .highlight_null(color="#f1f1f1")  # Color NaNs grey
    .format(precision=2)
)

,no_of_dependents,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value
no_of_dependents,1.00,0.01,-0.00,-0.02,-0.01,0.01,-0.00,0.00,0.01
income_annum,0.01,1.00,0.93,0.01,-0.02,0.64,0.64,0.93,0.85
loan_amount,-0.00,0.93,1.00,0.01,-0.02,0.59,0.60,0.86,0.79
loan_term,-0.02,0.01,0.01,1.00,0.01,0.01,-0.01,0.01,0.02
cibil_score,-0.01,-0.02,-0.02,0.01,1.00,-0.02,-0.00,-0.03,-0.02
residential_assets_value,0.01,0.64,0.59,0.01,-0.02,1.00,0.41,0.59,0.53
commercial_assets_value,-0.00,0.64,0.60,-0.01,-0.00,0.41,1.00,0.59,0.55
luxury_assets_value,0.00,0.93,0.86,0.01,-0.03,0.59,0.59,1.00,0.79
bank_asset_value,0.01,0.85,0.79,0.02,-0.02,0.53,0.55,0.79,1.00


In [19]:
s = corr.unstack()
so = s.sort_values(kind="quicksort", ascending=False)
so.to_frame("correlation").query(
    "(correlation >= 0.9) & (correlation != 1.0) "
).drop_duplicates("correlation").sort_index(level=0).reset_index().head()

,level_0,level_1,correlation
0,income_annum,loan_amount,0.927470
1,income_annum,luxury_assets_value,0.929145
